In [33]:
import matplotlib.pyplot as plt
import torch
import torch.nn.utils.prune as prune
import numpy as np
from transformers import RobertaConfig, RobertaModel
from transformers import GPT2Tokenizer, GPT2Model
from transformers import T5Tokenizer, T5Model
from transformers import BartTokenizer, BartModel

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2022-11-07 23:01:48.377348: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-07 23:01:48.657007: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-07 23:01:48.741931: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'li

In [34]:
def get_children(m): 
    child = [c for c in m.children()]
    if len(child) > 0: 
        res = []
        for c in child:
            for x in get_children(c):
                res.append(x)
        return res
    else:
        return [m]

In [35]:
def get_prune_layers_no_bias(model):
    to_prune = []
    layers = get_children(model)
    for l in layers:
        p = l.named_parameters()
        for obj in p:
            if obj[0] == 'weight' or obj[0] == 'weight_orig':
                to_prune.append((l,obj[0]))
                
    return to_prune
                

def get_prune_layers_with_bias(model):
    to_prune = []
    layers = get_children(model)
    for l in layers:
        p = l.named_parameters()
        for obj in p:
            to_prune.append((l,obj[0]))
            
    return to_prune

In [36]:
def full_model_prune(model, sparsity):
#     model.toggle_pruning(True)
    to_prune = get_prune_layers_no_bias(model)
    print(to_prune)
    
    prune.global_unstructured(
        to_prune,
        pruning_method=prune.L1Unstructured,
        amount=sparsity,)
    
    print('finished prune')
    for (m,name) in to_prune:
        print('in remove')
        print(list(m.named_buffers()))
        prune.remove(m, name)

    
    return model

In [ ]:
sparsity_values = [0.1,0.5,0.9,0.95,0.99]
# sparsity_values = [0.1]

for sparsity in sparsity_values: 
    print(sparsity)
    model = BartModel.from_pretrained('facebook/bart-base')
    model = full_model_prune(model,sparsity)
    model.save_pretrained(f"bartmodels/bart_{sparsity}")

model = BartModel.from_pretrained('facebook/bart-base')
model.save_pretrained(f"bartmodels/bart_base")                          
for sparsity in sparsity_values:
    print(sparsity)
    model = GPT2Model.from_pretrained("gpt2")
    model.config.pad_token_id = model.config.eos_token_id
    model = full_model_prune(model,sparsity)   
    model.save_pretrained(f"gpt2models/gpt2_{sparsity}")

model = GPT2Model.from_pretrained("gpt2")
m = full_model_prune(model,0.1)
print(m)
model = GPT2Model.from_pretrained("gpt2")
model.config.pad_token_id = model.config.eos_token_id
model.save_pretrained(f"gpt2models/gpt2")
for sparsity in sparsity_values:
    model = T5Model.from_pretrained("t5-small")
    model = full_model_prune(model,sparsity)
    model.save_pretrained(f"t5models/t5_{sparsity}")
model = T5Model.from_pretrained("t5-small")
model.save_pretrained(f"t5models/t5_small")
for sparsity in sparsity_values:
    print(sparsity)
    roberta_model = RobertaModel.from_pretrained("roberta-base")
    roberta_model = full_model_prune(roberta_model, sparsity)
    roberta_model.save_pretrained(f"roberta_models/roberta_{sparsity}")
roberta_model = RobertaModel.from_pretrained("roberta-base") 
roberta_model.save_pretrained(f"roberta_models/roberta_base")
model = T5Model.from_pretrained("t5-base")
print(model)
    model = full_model_prune(model,sparsity)
    model.save_pretrained(f"t5models/t5_{sparsity}")

# ROBERTA SST2

In [11]:
!python3 transformers/examples/pytorch/text-classification/run_glue.py \
  --model_name_or_path ~/deep_learning/llm-sparsification-boser/src/roberta_models/roberta_0.1 \
  --task_name sst2 \
  --tokenizer_name "roberta-base" \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3.0 \
  --output_dir test-sst2-roberta_0.1/

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2022-11-07 22:11:42.754457: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-07 22:11:42.879454: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-07 22:11:42.883306: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'li

[INFO|tokenization_utils_base.py:1775] 2022-11-07 22:11:47,187 >> loading file vocab.json from cache at /home/ribhavb/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/vocab.json
[INFO|tokenization_utils_base.py:1775] 2022-11-07 22:11:47,187 >> loading file merges.txt from cache at /home/ribhavb/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/merges.txt
[INFO|tokenization_utils_base.py:1775] 2022-11-07 22:11:47,187 >> loading file tokenizer.json from cache at /home/ribhavb/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/tokenizer.json
[INFO|tokenization_utils_base.py:1775] 2022-11-07 22:11:47,187 >> loading file added_tokens.json from cache at None
[INFO|tokenization_utils_base.py:1775] 2022-11-07 22:11:47,187 >> loading file special_tokens_map.json from cache at None
[INFO|tokenization_utils_base.py:1775] 2022-11-07 22:11:47,187 >> loading file tok

  0%|                                       | 1/6315 [00:15<26:58:58, 15.38s/it]^C


In [19]:
!python3 transformers/examples/pytorch/text-classification/run_glue.py \
  --model_name_or_path ~/deep_learning/llm-sparsification-boser/src/roberta_models/roberta_0.5 \
  --task_name sst2 \
  --tokenizer_name "roberta-base" \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3.0 \
  --output_dir test-sst2-roberta_0.5/


/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2022-11-07 22:14:24.959373: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-07 22:14:25.077982: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-07 22:14:25.083109: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'li

[INFO|tokenization_utils_base.py:1775] 2022-11-07 22:14:29,119 >> loading file vocab.json from cache at /home/ribhavb/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/vocab.json
[INFO|tokenization_utils_base.py:1775] 2022-11-07 22:14:29,119 >> loading file merges.txt from cache at /home/ribhavb/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/merges.txt
[INFO|tokenization_utils_base.py:1775] 2022-11-07 22:14:29,119 >> loading file tokenizer.json from cache at /home/ribhavb/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/tokenizer.json
[INFO|tokenization_utils_base.py:1775] 2022-11-07 22:14:29,119 >> loading file added_tokens.json from cache at None
[INFO|tokenization_utils_base.py:1775] 2022-11-07 22:14:29,119 >> loading file special_tokens_map.json from cache at None
[INFO|tokenization_utils_base.py:1775] 2022-11-07 22:14:29,119 >> loading file tok

  0%|                                                  | 0/6315 [00:00<?, ?it/s]^C
Traceback (most recent call last):
  File "transformers/examples/pytorch/text-classification/run_glue.py", line 622, in <module>
    main()
  File "transformers/examples/pytorch/text-classification/run_glue.py", line 530, in main
    train_result = trainer.train(resume_from_checkpoint=checkpoint)
  File "/home/ribhavb/.local/lib/python3.8/site-packages/transformers/trainer.py", line 1515, in train
    return inner_training_loop(
  File "/home/ribhavb/.local/lib/python3.8/site-packages/transformers/trainer.py", line 1763, in _inner_training_loop
    tr_loss_step = self.training_step(model, inputs)
  File "/home/ribhavb/.local/lib/python3.8/site-packages/transformers/trainer.py", line 2522, in training_step
    loss = self.compute_loss(model, inputs)
  File "/home/ribhavb/.local/lib/python3.8/site-packages/transformers/trainer.py", line 2554, in compute_loss
    outputs = model(**inputs)
  File "/home/ribh

In [ ]:
!python3 transformers/examples/pytorch/text-classification/run_glue.py \
  --model_name_or_path ~/deep_learning/llm-sparsification-boser/src/roberta_models/roberta_0.9 \
  --task_name sst2 \
  --tokenizer_name "roberta-base" \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3.0 \
  --output_dir test-sst2-roberta_0.9/

In [ ]:
!python3 transformers/examples/pytorch/text-classification/run_glue.py \
  --model_name_or_path ~/deep_learning/llm-sparsification-boser/src/roberta_models/roberta_0.95 \
  --task_name sst2 \
  --tokenizer_name "roberta-base" \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3.0 \
  --output_dir test-sst2-roberta_0.95/

In [ ]:
!python3 transformers/examples/pytorch/text-classification/run_glue.py \
  --model_name_or_path ~/deep_learning/llm-sparsification-boser/src/roberta_models/roberta_0.99 \
  --task_name sst2 \
  --tokenizer_name "roberta-base" \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3.0 \
  --output_dir test-sst2-roberta_0.99/

In [20]:
!python3 transformers/examples/pytorch/text-classification/run_glue.py \
  --model_name_or_path ~/deep_learning/llm-sparsification-boser/src/roberta_models/roberta_base \
  --task_name sst2 \
  --tokenizer_name "roberta-base" \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3.0 \
  --output_dir test-sst2-roberta_base/

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2022-11-07 22:14:47.721431: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-07 22:14:47.852803: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-07 22:14:47.858520: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'li

[INFO|tokenization_utils_base.py:1775] 2022-11-07 22:14:51,828 >> loading file vocab.json from cache at /home/ribhavb/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/vocab.json
[INFO|tokenization_utils_base.py:1775] 2022-11-07 22:14:51,828 >> loading file merges.txt from cache at /home/ribhavb/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/merges.txt
[INFO|tokenization_utils_base.py:1775] 2022-11-07 22:14:51,828 >> loading file tokenizer.json from cache at /home/ribhavb/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/tokenizer.json
[INFO|tokenization_utils_base.py:1775] 2022-11-07 22:14:51,828 >> loading file added_tokens.json from cache at None
[INFO|tokenization_utils_base.py:1775] 2022-11-07 22:14:51,828 >> loading file special_tokens_map.json from cache at None
[INFO|tokenization_utils_base.py:1775] 2022-11-07 22:14:51,828 >> loading file tok

  0%|                                       | 7/6315 [01:54<28:17:26, 16.15s/it]^C
Traceback (most recent call last):
  File "transformers/examples/pytorch/text-classification/run_glue.py", line 622, in <module>
    main()
  File "transformers/examples/pytorch/text-classification/run_glue.py", line 530, in main
    train_result = trainer.train(resume_from_checkpoint=checkpoint)
  File "/home/ribhavb/.local/lib/python3.8/site-packages/transformers/trainer.py", line 1515, in train
    return inner_training_loop(
  File "/home/ribhavb/.local/lib/python3.8/site-packages/transformers/trainer.py", line 1763, in _inner_training_loop
    tr_loss_step = self.training_step(model, inputs)
  File "/home/ribhavb/.local/lib/python3.8/site-packages/transformers/trainer.py", line 2522, in training_step
    loss = self.compute_loss(model, inputs)
  File "/home/ribhavb/.local/lib/python3.8/site-packages/transformers/trainer.py", line 2554, in compute_loss
    outputs = model(**inputs)
  File "/home/ribh

# ROBERTA COLA 

In [21]:
!python3 transformers/examples/pytorch/text-classification/run_glue.py \
  --model_name_or_path ~/deep_learning/llm-sparsification-boser/src/roberta_models/roberta_0.1 \
  --task_name cola \
  --tokenizer_name "roberta-base" \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3.0 \
  --output_dir test-cola-roberta_0.1/

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2022-11-07 22:16:58.543165: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-07 22:16:58.897091: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-07 22:16:58.937640: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'li

[INFO|tokenization_utils_base.py:1775] 2022-11-07 22:17:06,003 >> loading file vocab.json from cache at /home/ribhavb/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/vocab.json
[INFO|tokenization_utils_base.py:1775] 2022-11-07 22:17:06,003 >> loading file merges.txt from cache at /home/ribhavb/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/merges.txt
[INFO|tokenization_utils_base.py:1775] 2022-11-07 22:17:06,003 >> loading file tokenizer.json from cache at /home/ribhavb/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/tokenizer.json
[INFO|tokenization_utils_base.py:1775] 2022-11-07 22:17:06,003 >> loading file added_tokens.json from cache at None
[INFO|tokenization_utils_base.py:1775] 2022-11-07 22:17:06,003 >> loading file special_tokens_map.json from cache at None
[INFO|tokenization_utils_base.py:1775] 2022-11-07 22:17:06,003 >> loading file tok

  0%|                                         | 1/804 [00:17<3:58:40, 17.83s/it]^C


In [ ]:
!python3 transformers/examples/pytorch/text-classification/run_glue.py \
  --model_name_or_path ~/deep_learning/llm-sparsification-boser/src/roberta_models/roberta_0.5 \
  --task_name cola \
  --tokenizer_name "roberta-base" \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3.0 \
  --output_dir test-cola-roberta_0.5/

In [ ]:
!python3 transformers/examples/pytorch/text-classification/run_glue.py \
  --model_name_or_path ~/deep_learning/llm-sparsification-boser/src/roberta_models/roberta_0.9 \
  --task_name cola \
  --tokenizer_name "roberta-base" \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3.0 \
  --output_dir test-cola-roberta_0.9/

In [ ]:
!python3 transformers/examples/pytorch/text-classification/run_glue.py \
  --model_name_or_path ~/deep_learning/llm-sparsification-boser/src/roberta_models/roberta_0.95 \
  --task_name cola \
  --tokenizer_name "roberta-base" \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3.0 \
  --output_dir test-cola-roberta_0.95/

In [ ]:
!python3 transformers/examples/pytorch/text-classification/run_glue.py \
  --model_name_or_path ~/deep_learning/llm-sparsification-boser/src/roberta_models/roberta_0.99 \
  --task_name cola \
  --tokenizer_name "roberta-base" \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3.0 \
  --output_dir test-cola-roberta_0.99/

In [3]:
!python3 transformers/examples/pytorch/text-classification/run_glue.py \
  --model_name_or_path ~/deep_learning/llm-sparsification-boser/src/roberta_models/roberta_base \
  --task_name cola \
  --tokenizer_name "roberta-base" \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3.0 \
  --output_dir test-cola-roberta_base/

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2022-11-07 22:06:12.255635: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-07 22:06:12.628069: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-07 22:06:12.668195: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'li

[INFO|tokenization_utils_base.py:1775] 2022-11-07 22:06:19,355 >> loading file vocab.json from cache at /home/ribhavb/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/vocab.json
[INFO|tokenization_utils_base.py:1775] 2022-11-07 22:06:19,356 >> loading file merges.txt from cache at /home/ribhavb/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/merges.txt
[INFO|tokenization_utils_base.py:1775] 2022-11-07 22:06:19,356 >> loading file tokenizer.json from cache at /home/ribhavb/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/tokenizer.json
[INFO|tokenization_utils_base.py:1775] 2022-11-07 22:06:19,356 >> loading file added_tokens.json from cache at None
[INFO|tokenization_utils_base.py:1775] 2022-11-07 22:06:19,356 >> loading file special_tokens_map.json from cache at None
[INFO|tokenization_utils_base.py:1775] 2022-11-07 22:06:19,356 >> loading file tok

  0%|                                         | 1/804 [00:16<3:40:51, 16.50s/it]^C
Traceback (most recent call last):
  File "transformers/examples/pytorch/text-classification/run_glue.py", line 622, in <module>
    main()
  File "transformers/examples/pytorch/text-classification/run_glue.py", line 530, in main
    train_result = trainer.train(resume_from_checkpoint=checkpoint)
  File "/home/ribhavb/.local/lib/python3.8/site-packages/transformers/trainer.py", line 1515, in train
    return inner_training_loop(
  File "/home/ribhavb/.local/lib/python3.8/site-packages/transformers/trainer.py", line 1830, in _inner_training_loop
    self.optimizer.step()
  File "/home/ribhavb/.local/lib/python3.8/site-packages/torch/optim/lr_scheduler.py", line 65, in wrapper
    return wrapped(*args, **kwargs)
  File "/home/ribhavb/.local/lib/python3.8/site-packages/torch/optim/optimizer.py", line 113, in wrapper
    return func(*args, **kwargs)
  File "/home/ribhavb/.local/lib/python3.8/site-packages/tr

# GPT 2 COLA

In [23]:
!python3 transformers/examples/pytorch/text-classification/run_glue.py \
  --model_name_or_path ~/deep_learning/llm-sparsification-boser/src/gpt2models/gpt2 \
  --task_name cola \
  --tokenizer_name ~/deep_learning/llm-sparsification-boser/src/gpt2models/gpt2_tokenizer \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3.0 \
  --output_dir test-cola-gpt2/

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2022-11-07 22:19:48.311144: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-07 22:19:48.625631: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-07 22:19:48.703293: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'li

In [ ]:
!python3 transformers/examples/pytorch/text-classification/run_glue.py \
  --model_name_or_path ~/deep_learning/llm-sparsification-boser/src/gpt2models/gpt2_0.1 \
  --task_name cola \
  --tokenizer_name ~/deep_learning/llm-sparsification-boser/src/gpt2models/gpt2_tokenizer \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3.0 \
  --output_dir test-cola-gpt2_0.1/

In [ ]:
!python3 transformers/examples/pytorch/text-classification/run_glue.py \
  --model_name_or_path ~/deep_learning/llm-sparsification-boser/src/gpt2models/gpt2_0.5 \
  --task_name cola \
  --tokenizer_name ~/deep_learning/llm-sparsification-boser/src/gpt2models/gpt2_tokenizer \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3.0 \
  --output_dir test-cola-gpt2_0.5/

In [ ]:
!python3 transformers/examples/pytorch/text-classification/run_glue.py \
  --model_name_or_path ~/deep_learning/llm-sparsification-boser/src/gpt2models/gpt2_0.9 \
  --task_name cola \
  --tokenizer_name ~/deep_learning/llm-sparsification-boser/src/gpt2models/gpt2_tokenizer \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3.0 \
  --output_dir test-cola-gpt2_0.9/

In [ ]:
!python3 transformers/examples/pytorch/text-classification/run_glue.py \
  --model_name_or_path ~/deep_learning/llm-sparsification-boser/src/gpt2models/gpt2_0.95 \
  --task_name cola \
  --tokenizer_name ~/deep_learning/llm-sparsification-boser/src/gpt2models/gpt2_tokenizer \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3.0 \
  --output_dir test-cola-gpt2_0.95/

In [24]:
!python3 transformers/examples/pytorch/text-classification/run_glue.py \
  --model_name_or_path ~/deep_learning/llm-sparsification-boser/src/gpt2models/gpt2_0.99 \
  --task_name cola \
  --tokenizer_name ~/deep_learning/llm-sparsification-boser/src/gpt2models/gpt2_tokenizer \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3.0 \
  --output_dir test-cola-gpt2_0.99/

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2022-11-07 22:21:31.652449: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-07 22:21:31.769147: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-07 22:21:31.772675: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'li

[INFO|modeling_utils.py:2609] 2022-11-07 22:21:36,623 >> All model checkpoint weights were used when initializing GPT2ForSequenceClassification.

[WARNING|modeling_utils.py:2611] 2022-11-07 22:21:36,623 >> Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at /home/ribhavb/deep_learning/llm-sparsification-boser/src/gpt2models/gpt2_0.99 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Running tokenizer on dataset:  50%|███████▌       | 1/2 [00:00<00:00, 13.94ba/s]
INFO:__main__:Sample 1824 of the training set: {'sentence': 'I acknowledged that my father, he was tight as an owl.', 'label': 0, 'idx': 1824, 'input_ids': [40, 10810, 326, 616, 2988, 11, 339, 373, 5381, 355, 281, 39610, 13, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256

# GPT2 sst2

In [25]:
!python3 transformers/examples/pytorch/text-classification/run_glue.py \
  --model_name_or_path ~/deep_learning/llm-sparsification-boser/src/gpt2models/gpt2 \
  --task_name sst2 \
  --tokenizer_name ~/deep_learning/llm-sparsification-boser/src/gpt2models/gpt2_tokenizer \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3.0 \
  --output_dir test-sst2-gpt2/

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2022-11-07 22:23:00.584040: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-07 22:23:00.973214: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-07 22:23:01.123953: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'li

[INFO|modeling_utils.py:2609] 2022-11-07 22:23:09,516 >> All model checkpoint weights were used when initializing GPT2ForSequenceClassification.

[WARNING|modeling_utils.py:2611] 2022-11-07 22:23:09,516 >> Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at /home/ribhavb/deep_learning/llm-sparsification-boser/src/gpt2models/gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Running tokenizer on dataset:  50%|███████▌       | 1/2 [00:00<00:00, 12.64ba/s]
INFO:__main__:Sample 14592 of the training set: {'sentence': 'a great movie ', 'label': 1, 'idx': 14592, 'input_ids': [64, 1049, 3807, 220, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 5025

In [26]:
!python3 transformers/examples/pytorch/text-classification/run_glue.py \
  --model_name_or_path ~/deep_learning/llm-sparsification-boser/src/gpt2models/gpt2_0.1 \
  --task_name sst2 \
  --tokenizer_name ~/deep_learning/llm-sparsification-boser/src/gpt2models/gpt2_tokenizer \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3.0 \
  --output_dir test-sst2-gpt2_0.1/

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2022-11-07 22:24:01.998346: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-07 22:24:02.356555: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-07 22:24:02.468658: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'li

[INFO|modeling_utils.py:2609] 2022-11-07 22:24:09,856 >> All model checkpoint weights were used when initializing GPT2ForSequenceClassification.

[WARNING|modeling_utils.py:2611] 2022-11-07 22:24:09,856 >> Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at /home/ribhavb/deep_learning/llm-sparsification-boser/src/gpt2models/gpt2_0.1 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Running tokenizer on dataset:   0%|                       | 0/1 [00:00<?, ?ba/s]
INFO:__main__:Sample 14592 of the training set: {'sentence': 'a great movie ', 'label': 1, 'idx': 14592, 'input_ids': [64, 1049, 3807, 220, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 

In [ ]:
!python3 transformers/examples/pytorch/text-classification/run_glue.py \
  --model_name_or_path ~/deep_learning/llm-sparsification-boser/src/gpt2models/gpt2_0.5 \
  --task_name sst2 \
  --tokenizer_name ~/deep_learning/llm-sparsification-boser/src/gpt2models/gpt2_tokenizer \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3.0 \
  --output_dir test-sst2-gpt2_0.5/

In [ ]:
!python3 transformers/examples/pytorch/text-classification/run_glue.py \
  --model_name_or_path ~/deep_learning/llm-sparsification-boser/src/gpt2models/gpt2_0.9 \
  --task_name sst2 \
  --tokenizer_name ~/deep_learning/llm-sparsification-boser/src/gpt2models/gpt2_tokenizer \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3.0 \
  --output_dir test-sst2-gpt2_0.9/

In [ ]:
!python3 transformers/examples/pytorch/text-classification/run_glue.py \
  --model_name_or_path ~/deep_learning/llm-sparsification-boser/src/gpt2models/gpt2_0.95 \
  --task_name sst2 \
  --tokenizer_name ~/deep_learning/llm-sparsification-boser/src/gpt2models/gpt2_tokenizer \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3.0 \
  --output_dir test-sst2-gpt2_0.95/

In [ ]:
!python3 transformers/examples/pytorch/text-classification/run_glue.py \
  --model_name_or_path ~/deep_learning/llm-sparsification-boser/src/gpt2models/gpt2_0.99 \
  --task_name sst2 \
  --tokenizer_name ~/deep_learning/llm-sparsification-boser/src/gpt2models/gpt2_tokenizer \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3.0 \
  --output_dir test-sst2-gpt2_0.99/

# BART COLA

In [31]:
!python3 transformers/examples/pytorch/text-classification/run_glue.py \
  --model_name_or_path ~/deep_learning/llm-sparsification-boser/src/bartmodels/bart_base \
  --task_name cola \
  --tokenizer_name "facebook/bart-base" \
  --overwrite_output_dir \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3.0 \
  --output_dir test-cola-bart_base/

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2022-11-07 22:44:44.155239: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-07 22:44:44.272183: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-07 22:44:44.275803: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'li

[INFO|tokenization_auto.py:427] 2022-11-07 22:44:47,889 >> Could not locate the tokenizer configuration file, will try to use the model config instead.
[INFO|configuration_utils.py:654] 2022-11-07 22:44:48,083 >> loading configuration file config.json from cache at /home/ribhavb/.cache/huggingface/hub/models--facebook--bart-base/snapshots/84358834e73de6a82c22cec1d90eb45ef4f6eba5/config.json
[INFO|configuration_utils.py:706] 2022-11-07 22:44:48,084 >> Model config BartConfig {
  "_name_or_path": "facebook/bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "en

[INFO|trainer.py:725] 2022-11-07 22:44:51,024 >> The following columns in the training set don't have a corresponding argument in `BartForSequenceClassification.forward` and have been ignored: idx, sentence. If idx, sentence are not expected by `BartForSequenceClassification.forward`,  you can safely ignore this message.
/home/ribhavb/.local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
[INFO|trainer.py:1622] 2022-11-07 22:44:51,032 >> ***** Running training *****
[INFO|trainer.py:1623] 2022-11-07 22:44:51,032 >>   Num examples = 8551
[INFO|trainer.py:1624] 2022-11-07 22:44:51,032 >>   Num Epochs = 3
[INFO|trainer.py:1625] 2022-11-07 22:44:51,032 >>   Instantaneous batch size per device = 32
[INFO|trainer.py:1626] 2022-11-07 22:44:51,033 >

In [ ]:
!python3 transformers/examples/pytorch/text-classification/run_glue.py \
  --model_name_or_path ~/deep_learning/llm-sparsification-boser/src/bartmodels/bart_0.1 \
  --task_name cola \
  --tokenizer_name "facebook/bart-base" \
  --overwrite_output_dir \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3.0 \
  --output_dir test-cola-bart_0.1/

In [ ]:
!python3 transformers/examples/pytorch/text-classification/run_glue.py \
  --model_name_or_path ~/deep_learning/llm-sparsification-boser/src/bartmodels/bart_0.5 \
  --task_name cola \
  --tokenizer_name "facebook/bart-base" \
  --overwrite_output_dir \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3.0 \
  --output_dir test-cola-bart_0.5/

In [ ]:
!python3 transformers/examples/pytorch/text-classification/run_glue.py \
  --model_name_or_path ~/deep_learning/llm-sparsification-boser/src/bartmodels/bart_0.9 \
  --task_name cola \
  --tokenizer_name "facebook/bart-base" \
  --overwrite_output_dir \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3.0 \
  --output_dir test-cola-bart_0.9/

In [ ]:
!python3 transformers/examples/pytorch/text-classification/run_glue.py \
  --model_name_or_path ~/deep_learning/llm-sparsification-boser/src/bartmodels/bart_0.95 \
  --task_name cola \
  --tokenizer_name "facebook/bart-base" \
  --overwrite_output_dir \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3.0 \
  --output_dir test-cola-bart_0.95/

In [ ]:
!python3 transformers/examples/pytorch/text-classification/run_glue.py \
  --model_name_or_path ~/deep_learning/llm-sparsification-boser/src/bartmodels/bart_0.99 \
  --task_name cola \
  --tokenizer_name "facebook/bart-base" \
  --overwrite_output_dir \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3.0 \
  --output_dir test-cola-bart_0.99/

# BART SST2

In [32]:
!python3 transformers/examples/pytorch/text-classification/run_glue.py \
  --model_name_or_path ~/deep_learning/llm-sparsification-boser/src/bartmodels/bart_base \
  --task_name sst2 \
  --tokenizer_name "facebook/bart-base" \
  --overwrite_output_dir \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3.0 \
  --output_dir test-sst2-bart_base/

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2022-11-07 22:46:33.739231: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-07 22:46:34.110750: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-07 22:46:34.197177: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'li

[INFO|tokenization_auto.py:427] 2022-11-07 22:46:41,316 >> Could not locate the tokenizer configuration file, will try to use the model config instead.
[INFO|configuration_utils.py:654] 2022-11-07 22:46:41,579 >> loading configuration file config.json from cache at /home/ribhavb/.cache/huggingface/hub/models--facebook--bart-base/snapshots/84358834e73de6a82c22cec1d90eb45ef4f6eba5/config.json
[INFO|configuration_utils.py:706] 2022-11-07 22:46:41,580 >> Model config BartConfig {
  "_name_or_path": "facebook/bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "en

[INFO|trainer.py:725] 2022-11-07 22:46:47,554 >> The following columns in the training set don't have a corresponding argument in `BartForSequenceClassification.forward` and have been ignored: idx, sentence. If idx, sentence are not expected by `BartForSequenceClassification.forward`,  you can safely ignore this message.
/home/ribhavb/.local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
[INFO|trainer.py:1622] 2022-11-07 22:46:47,561 >> ***** Running training *****
[INFO|trainer.py:1623] 2022-11-07 22:46:47,561 >>   Num examples = 67349
[INFO|trainer.py:1624] 2022-11-07 22:46:47,561 >>   Num Epochs = 3
[INFO|trainer.py:1625] 2022-11-07 22:46:47,561 >>   Instantaneous batch size per device = 32
[INFO|trainer.py:1626] 2022-11-07 22:46:47,561 

In [ ]:
!python3 transformers/examples/pytorch/text-classification/run_glue.py \
  --model_name_or_path ~/deep_learning/llm-sparsification-boser/src/bartmodels/bart_0.1 \
  --task_name sst2 \
  --tokenizer_name "facebook/bart-base" \
  --overwrite_output_dir \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3.0 \
  --output_dir test-sst2-bart_0.1/

In [ ]:
!python3 transformers/examples/pytorch/text-classification/run_glue.py \
  --model_name_or_path ~/deep_learning/llm-sparsification-boser/src/bartmodels/bart_0.5 \
  --task_name sst2 \
  --tokenizer_name "facebook/bart-base" \
  --overwrite_output_dir \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3.0 \
  --output_dir test-sst2-bart_0.5/

In [ ]:
!python3 transformers/examples/pytorch/text-classification/run_glue.py \
  --model_name_or_path ~/deep_learning/llm-sparsification-boser/src/bartmodels/bart_0.9 \
  --task_name sst2 \
  --tokenizer_name "facebook/bart-base" \
  --overwrite_output_dir \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3.0 \
  --output_dir test-sst2-bart_0.9/

In [ ]:
!python3 transformers/examples/pytorch/text-classification/run_glue.py \
  --model_name_or_path ~/deep_learning/llm-sparsification-boser/src/bartmodels/bart_0.95 \
  --task_name sst2 \
  --tokenizer_name "facebook/bart-base" \
  --overwrite_output_dir \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3.0 \
  --output_dir test-sst2-bart_0.95/

In [ ]:
!python3 transformers/examples/pytorch/text-classification/run_glue.py \
  --model_name_or_path ~/deep_learning/llm-sparsification-boser/src/bartmodels/bart_0.99 \
  --task_name sst2 \
  --tokenizer_name "facebook/bart-base" \
  --overwrite_output_dir \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3.0 \
  --output_dir test-sst2-bart_0.99/